![Gliese Catalog HR Diagram](https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/HRDiagram_es.svg/1200px-HRDiagram_es.svg.png)

# Diagrama Hertzsprung-Russell del Catálogo Gliese
**Análisis de etapas evolutivas estelares usando Machine Learning**

---
Última actualización: [2025-08-07]

In [ ]:
%%capture
!pip install astroquery scikit-learn matplotlib astropy

## 1. Obtención de datos

In [ ]:
from astroquery.vizier import Vizier
import pandas as pd
import numpy as np

# Configurar consulta al catálogo actualizado
Vizier.ROW_LIMIT = -1
catalogs = Vizier.get_catalogs('V/137D')

# Convertir a DataFrame
gliese_df = catalogs[0].to_pandas()

## 2. Preprocesamiento

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Filtrar columnas relevantes
features = gliese_df[['Teff', 'logL']].dropna()

# Normalización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

## 3. Clasificación evolutiva

In [ ]:
# Determinación óptima de clusters (método del codo)
wcss = []
for i in range(1, 6):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

In [ ]:
# Aplicar K-Means con 4 clusters (principales etapas)
optimal_clusters = 4
kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', random_state=42)
clusters = kmeans.fit_predict(X_scaled)

## 4. Visualización

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(12, 8))

# Paleta de colores para etapas evolutivas
cmap = ListedColormap(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])

scatter = plt.scatter(features['Teff'], features['logL'],
                      c=clusters, cmap=cmap, s=15, alpha=0.8)

# Configuración del diagrama HR
plt.gca().invert_xaxis()
plt.colorbar(scatter, label='Etapa evolutiva')
plt.title('Diagrama Hertzsprung-Russell - Catálogo Gliese', fontsize=14)
plt.xlabel('Temperatura efectiva (K)', fontsize=12)
plt.ylabel('Luminosidad (log L/L☉)', fontsize=12)

# Anotaciones de etapas
stage_labels = {
    0: 'Secuencia principal',
    1: 'Gigantes',
    2: 'Enanas blancas',
    3: 'Subgigantes'
}
handles = [plt.Line2D([0], [0], marker='o', color='w', 
                      markerfacecolor=cmap(i), markersize=10)
           for i in range(optimal_clusters)]
plt.legend(handles, stage_labels.values(), title='Clasificación')

plt.show()

## Interpretación de clusters

| Cluster | Características                     | Etapa evolutiva típica |
|---------|-------------------------------------|------------------------|
| 0       | Alta temperatura, media luminosidad | Secuencia principal    |
| 1       | Media temperatura, alta luminosidad | Gigantes               |
| 2       | Alta temperatura, baja luminosidad  | Enanas blancas         |
| 3       | Media-alta temperatura, luminosidad intermedia | Subgigantes |